In [2]:
from datetime import date, timedelta
from dotenv import load_dotenv
from ipywidgets import (
    interact, interactive, fixed, interact_manual, Layout
)
import ipywidgets as widgets
import json
import os
import time
import requests
import pandas as pd

load_dotenv()

TOKEN = os.getenv('TOKEN')

HEADERS = {"Authorization": f"Bearer {TOKEN}"}

GRAPHQL_URL = "https://api.github.com/graphql"

In [3]:
start_date = widgets.DatePicker(
    value=date.today() - timedelta(days=365),
    description='Start Date',
    disabled=False
)
end_date = widgets.DatePicker(
    value=date.today() - timedelta(days=1),
    description='End Date',
    disabled=False
)

def fdate(start, end):
    print(
        "\nstart date: {}, end date: {}\n".format(start, end)
    )

date_out = widgets.interactive_output(
    fdate, {
        "start": start_date,
        "end": end_date
    }
)

display(start_date, end_date, date_out)

DatePicker(value=datetime.date(2024, 7, 18), description='Start Date', step=1)

DatePicker(value=datetime.date(2025, 7, 17), description='End Date', step=1)

Output()

In [4]:
def get_unique_commit_authors_between_dates(since, until):
    """
    > Endpoint #1
    
    - Accepts: A start and end date.
    
    - Returns: A list of unique commit authors within the date constraints supplied.
    """
    print(f"\nSearching for commit authors since {since} until {until}\n")
    q = (
        "{ rateLimit { limit cost remaining resetAt } "
        " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
        " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
        f" history( first: 50 since: \"{since}\" until: \"{until}\") "
        " { pageInfo { hasNextPage startCursor endCursor } "
        " edges { node { author { name email }}}}}}}}}"
    )
    try:
        authors = set()
        while q is not None:
            r = requests.post(
                GRAPHQL_URL,
                json={"query": q},
                headers=HEADERS
            )
            print(r.status_code)
            if r.status_code == 429:
                time.sleep(60)
            if r.status_code != 200:
                print("uh oh")
                raise
            response = r.json()
            print(response)
            commits = response["data"]["repository"]["ref"]["target"]["history"]["edges"]
            print(f"Found {len(commits)} commits since {since}")
            for commit in commits:
                authors.add(commit["node"]["author"]["email"])
            if response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["hasNextPage"] == True:
                after = response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["endCursor"]
                q = (
                    "{ rateLimit { limit cost remaining resetAt } "
                    " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
                    " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
                    f" history( first: 50 after {after}) "
                    " { pageInfo { hasNextPage startCursor endCursor } "
                    " edges { node { author { name email }}}}}}}}}"
                )
            else:
                q = None
        print(f"\n*** Unique authors: \n\n{authors}\n\n")
        return authors
    except Exception as e:
        print(e)
        raise

unique_authors = get_unique_commit_authors_between_dates(
    f"{start_date.value}T00:00:00",
    f"{end_date.value}T00:00:00"
)


Searching for commit authors since 2025-07-16T00:00:00 until 2025-07-18T00:00:00

200
{'data': {'rateLimit': {'limit': 5000, 'cost': 1, 'remaining': 4992, 'resetAt': '2025-07-18T19:50:16Z'}, 'repository': {'ref': {'target': {'history': {'pageInfo': {'hasNextPage': False, 'startCursor': 'a4966bc299919426fd77e5272bb7aac6f30ab73c 0', 'endCursor': 'a4966bc299919426fd77e5272bb7aac6f30ab73c 6'}, 'edges': [{'node': {'author': {'name': 'Vapre', 'email': 'anvilvapre@users.noreply.github.com'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'darkademic', 'email': '41052878+darkademic@users.noreply.github.com'}}}, {'node': {'author': {'name': 'darkademic',

In [5]:
df = pd.DataFrame([(a,) for a in unique_authors], columns=["author"])
df
                  

,author
0,anvilvapre@users.noreply.github.com
1,ashleynewson@smartsim.org.uk
2,matthias@mailaender.name
3,41052878+darkademic@users.noreply.github.com
